In [ ]:
from csv import DictReader

flows = []
with open('../../data/navigo_all_flows.csv', newline='') as csvfile:
    reader = DictReader(csvfile)
    for row in reader:
        if \
                row['destination_fr'] == 'Marseille' \
            and row['indate_fixed'].split('-')[0] == '1789' \
            :
            flows.append(dict(row))

print(len(flows))

In [ ]:
flows[0]

In [ ]:
set([f['source_suite'] for f in flows])

In [ ]:
flows_sourced = [f for f in flows if f['source_suite'] != 'G5']
print(len(flows_sourced))

Attention : on a pas filtré les net_route_marker = Q , il peut y avoir des doublons entre petit cabotage et la santé, A NETTOYER

In [ ]:
set([f['departure_state_fr'] for f in flows])

In [ ]:
state_to_partner = {'': 'Inconnu',
 'Autriche': 'Flandre et autres états de l\'Empereur',
 'Duché de Massa et Carrare': 'Italie',
 'Empire ottoman':'Levant et Barbarie',
 'Espagne': 'Espagne',
 'Etats pontificaux': 'Italie',
 'France': 'France',
 'Grande-Bretagne': 'Angleterre',
 'Malte': 'Italie',
 'Maroc': 'Levant et Barbarie',
 'Monaco': 'Italie',
 'Royaume de Naples': 'Italie',
 'Royaume de Piémont-Sardaigne': 'Italie',
 'République de Gênes': 'Italie',
 'République de Lucques': 'Italie',
 'République de Venise': 'Italie',
 'Toscane': 'Italie',
 'multi-Etat': 'Inconnu',
 'zone maritime': 'Inconnu'}


In [ ]:
from collections import Counter

departures = Counter()

for flow in flows_sourced:
    state = flow['departure_state_fr']
    partner = state_to_partner[state]
    departures.update({partner: 1})

print(departures)

In [ ]:
from csv import DictReader
from collections import Counter
import requests
import csv

# 1. get online csv data
def get_online_csv(url):
  results = []
  with requests.Session() as s:
      download = s.get(url)
      decoded_content = download.content.decode('utf-8')
      reader = csv.DictReader(decoded_content.splitlines(), delimiter=',')
      for row in reader:
        results.append(dict(row))
  return results

TONNAGE_SPREADSHEET_URL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTYdeIwpzaVpY_KS91cXiHxb309iYBS4JN_1_hW-_oyeysuwcIpC2VJ5fWeZJl4tA/pub?output=csv'

tonnage_data = get_online_csv(TONNAGE_SPREADSHEET_URL)

# 2. build a shipclass->tx dict
tonnage_estimates = {}
for l in tonnage_data:
    estimation = l['tonnage_estime_en_tx'] or 0
    if l['tonnage_estime_en_tx'] == 'No data':
        estimation = 0
    else :
        estimation = int(estimation)
    
    ship_class = l['ship_class']
    
    tonnage_estimates[ship_class] = estimation
   
tonnage_estimates[''] = 0
print(tonnage_estimates)

Attention : quand pas de classe de navire, tonnage égale 0 -> pas bien, INCERTAIN

In [ ]:
departures_tonnage = Counter()

for flow in flows_sourced:
    state = flow['departure_state_fr']
    partner = state_to_partner[state]
    departures_tonnage.update({partner: tonnage_estimates[flow['ship_class_standardized']]})

print(departures_tonnage)

In [ ]:
angleterre_tonnage = Counter()

for flow in flows_sourced:
    state = flow['departure_state_fr']
    if state == 'Grande-Bretagne':
        ship_class_standardized = flow['ship_class_standardized']
        angleterre_tonnage.update({ship_class_standardized: 1})

print(angleterre_tonnage)

In [ ]:
departures_tonnage_liste = [{"state" : state, "tonnage": tonnage} for state,tonnage in dict(departures_tonnage).items()]
departures_tonnage_liste

In [ ]:
from IPython.display import display


def VegaLite(spec):
    bundle = {}
    bundle['application/vnd.vegalite.v4+json'] = spec
    display(bundle, raw=True)

In [ ]:

VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Tonnage par partenaires, pour les entrées à Marseille en 1789",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect" #type de visualisation (ici rectangle)
    },
    "data": {
        "values": departures_tonnage_liste #mettre la liste des données
    },
    "encoding": {
        "x": {
            "field": "tonnage",
            "type": "quantitative", #ou nominal 
            "title": "tonnage",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "state",
            "type": "nominal",
            "title": "partenaires grouping"
        },
    }
})

In [ ]:
from vega import VegaLite
import pandas as pd

df = pd.DataFrame([departures_tonnage_liste])

VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Tonnage par partenaires, pour les entrées à Marseille en 1789",
    "mark": {
        "tooltip": {
         "content": "data"
        },
        "type": "rect" #type de visualisation (ici rectangle)
    },
    "encoding": {
        "x": {
            "field": "tonnage",
            "type": "quantitative", #ou nominal 
            "title": "tonnage",
            "axis": {
                "orient": "top"
            }
        },
        "y": {
            "field": "state",
            "type": "nominal",
            "title": "partenaires grouping"
        },
    }
}, df)